In [ ]:
!pip install --user torch===1.5.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
try:
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

from torchbearer import Trial
from torchbearer import callbacks
from torch import nn
import torch.nn.functional as F
from torchbearer.callbacks.torch_scheduler import LambdaLR
from torchbearer.callbacks import Callback
import numpy as np
import h5py
from torch.utils.data import Dataset, DataLoader
import os
from time import time

In [ ]:
import torch
print(torch.__version__)

In [ ]:
class LRPrinter(Callback):
    def __init__(self, opt):
        self._opt = opt

    def on_start_epoch(self, state):
        print(self._opt.param_groups[0]['lr'])
        
@callbacks.on_end_epoch
def save_state(state):
    #if state.EPOCH % 5 == 0:
    epoch = state[torchbearer.EPOCH]
    torch.save(state[torchbearer.MODEL].state_dict(), f'H:/pretrained_sem/pretrained_sn_{epoch}.weights')
    print('model saved '+str(epoch))

In [ ]:
def scale_to_unit_cube(vertices):
    # Mean-centre
    vertices -= np.mean(vertices, axis=0)
    # Scale
    vertices /= 2*np.amax(np.abs(vertices))
    
def voxelise(vertices, k):
    vox_ids = np.empty(vertices.shape[0], dtype=int)
    
    d_ind = lambda d : np.floor(d * k) if d < 1 else k-1
    
    # Ensures input vertices x,y,z between [0,1]
    for i,v in enumerate(vertices + 0.5):
        vox_ids[i] = np.sum([d_ind(v[i])*k**i for i in range(3)])
                           
    return vox_ids

def get_voxel_xyz(vox_id, k):
    x=y=z = 0    
    
    z = vox_id // k**2
    vox_id -= z*k**2
    y = vox_id // k
    vox_id -= y*k
    x = vox_id
    
    return np.asarray([x,y,z]) / k

def shuffle_voxels(vertices, vox_ids, k):
    shuffled_vertices = np.empty_like(vertices)
    
    ids = np.asarray([i for i in range(k**3)])
    shuffled_ids = np.random.permutation(ids)
    id_map = dict(np.stack((ids, shuffled_ids), axis=1))
    
    for i,v in enumerate(vertices):
        # translate to origin voxel
        xyz_from = get_voxel_xyz(vox_ids[i], k)
        # translate to destination voxel
        xyz_to = get_voxel_xyz(id_map[vox_ids[i]], k)
        shuffled_vertices[i] = v - xyz_from + xyz_to
    
    return shuffled_vertices

In [ ]:
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None, dim6=True):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        if dim6 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, :3], k=k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2)
  
    return feature      # (batch_size, 2*num_dims, num_points, k)

class Transform_Net(nn.Module):
    def __init__(self, args):
        super(Transform_Net, self).__init__()
        self.args = args
        self.k = 3

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm1d(1024)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv1d(128, 1024, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))

        self.linear1 = nn.Linear(1024, 512, bias=False)
        self.bn3 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 256, bias=False)
        self.bn4 = nn.BatchNorm1d(256)

        self.transform = nn.Linear(256, 3*3)
        init.constant_(self.transform.weight, 0)
        init.eye_(self.transform.bias.view(3, 3))

    def forward(self, x):
        batch_size = x.size(0)

        x = self.conv1(x)                       # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x = self.conv2(x)                       # (batch_size, 64, num_points, k) -> (batch_size, 128, num_points, k)
        x = x.max(dim=-1, keepdim=False)[0]     # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = self.conv3(x)                       # (batch_size, 128, num_points) -> (batch_size, 1024, num_points)
        x = x.max(dim=-1, keepdim=False)[0]     # (batch_size, 1024, num_points) -> (batch_size, 1024)

        x = F.leaky_relu(self.bn3(self.linear1(x)), negative_slope=0.2)     # (batch_size, 1024) -> (batch_size, 512)
        x = F.leaky_relu(self.bn4(self.linear2(x)), negative_slope=0.2)     # (batch_size, 512) -> (batch_size, 256)

        x = self.transform(x)                   # (batch_size, 256) -> (batch_size, 3*3)
        x = x.view(batch_size, 3, 3)            # (batch_size, 3*3) -> (batch_size, 3, 3)

        return x

In [ ]:
class DGCNN_semseg(nn.Module):
    def __init__(self, k=20, emb_dims=1024, dropout=0.5, seg_num_all=27):
        super(DGCNN_semseg, self).__init__()
        self.k = k
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(64)
        self.bn5 = nn.BatchNorm2d(64)
        self.bn6 = nn.BatchNorm1d(emb_dims)
        self.bn7 = nn.BatchNorm1d(512)
        self.bn8 = nn.BatchNorm1d(256)

        self.conv1 = nn.Sequential(nn.Conv2d(12, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv6 = nn.Sequential(nn.Conv1d(192, emb_dims, kernel_size=1, bias=False),
                                   self.bn6,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv7 = nn.Sequential(nn.Conv1d(1216, 512, kernel_size=1, bias=False),
                                   self.bn7,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv8 = nn.Sequential(nn.Conv1d(512, 256, kernel_size=1, bias=False),
                                   self.bn8,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.dp1 = nn.Dropout(p=dropout)
        self.conv9 = nn.Conv1d(256, seg_num_all, kernel_size=1, bias=False)
        

    def forward(self, x):
        batch_size = x.size(0)
        num_points = x.size(2)

        x = get_graph_feature(x, k=self.k)   # (batch_size, 9, num_points) -> (batch_size, 9*2, num_points, k)
        x = self.conv1(x)                       # (batch_size, 9*2, num_points, k) -> (batch_size, 64, num_points, k)
        x = self.conv2(x)                       # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x = self.conv4(x)                       # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv5(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = torch.cat((x1, x2, x3), dim=1)      # (batch_size, 64*3, num_points)

        x = self.conv6(x)                       # (batch_size, 64*3, num_points) -> (batch_size, emb_dims, num_points)
        x = x.max(dim=-1, keepdim=True)[0]      # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims, 1)

        x = x.repeat(1, 1, num_points)          # (batch_size, 1024, num_points)
        x = torch.cat((x, x1, x2, x3), dim=1)   # (batch_size, 1024+64*3, num_points)

        x = self.conv7(x)                       # (batch_size, 1024+64*3, num_points) -> (batch_size, 512, num_points)
        x = self.conv8(x)                       # (batch_size, 512, num_points) -> (batch_size, 256, num_points)
        x = self.dp1(x)
        x = self.conv9(x)                       # (batch_size, 256, num_points) -> (batch_size, seg_num_all, num_points)
        
        return x.permute(0,2,1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.cuda.memory_allocated(device)/1e9)
print(torch.cuda.memory_cached(device)/1e9)
torch.cuda.empty_cache()
model=DGCNN_semseg()

print(torch.cuda.memory_allocated(device)/1e9)
print(torch.cuda.memory_cached(device)/1e9)
#model = nn.DataParallel(model)
#model.load_state_dict(torch.load('pretrained/model.1024.t7'))

# Training

In [ ]:
dataset = 'H:/shapenet57448xyzonly.npz'

with np.load(dataset) as sn:
    data = sn['data']

voxids = 'H:/voxids.npz'

with np.load(voxids) as lbs:
    voxids = lbs['arr_0']

In [ ]:
#!pip install torch-scatter==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
#!pip install torch-sparse==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
#!pip install torch-cluster==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
#!pip install torch-spline-conv==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
#from torch_geometric.datasets import S3DIS

In [ ]:
class ShapeNetDataset(Dataset):
    
    def __init__(self, models, voxids, num_points=1024):
        
        data, vids = torch.empty((len(models),num_points,3)), torch.empty((len(models),num_points), dtype=torch.long)
        
        for i, pointcloud in enumerate(models):
            
            inds = np.random.permutation(range(2048))
            vs = voxids[i,inds[:num_points]]
            
            shuffled = shuffle_voxels(pointcloud[inds[:num_points]], vs, 3)
            
            data[i] = torch.as_tensor(shuffled)
            vids[i] = torch.as_tensor(vs)
                    
        self.size   = len(vids)
        self.data   = torch.cat([data.permute(0,2,1), torch.rand(len(models),3,num_points)], 1)
        self.vids = vids
        print('Done')
          
    def __getitem__(self, index):
        if index >= len(self):
            raise IndexError(f'self.__class__.__name__ index out of range.')

        return self.data[index], self.vids[index]

    def __len__(self):
        return self.size

In [ ]:
with open('H:/trainloader.pkl','rb') as f:
    train_loader = torch.load(f)

In [ ]:
def loss_function(pred, gold, smoothing=True):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    
    pred = pred.contiguous().view(-1, pred.shape[-1])
    gold = gold.contiguous().flatten()

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)
        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.cross_entropy(pred, gold, reduction='mean')

    return loss

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)

# gamma = decay_rate in tf
# In the paper they clipped learning rates to 1e-5

def calculate_gamma_initial_lr(epoch):
    lr_initial = 0.001
    factor = 0.5**epoch
    lr = lr_initial*factor
    if lr < 0.00001:
        return 0.01
    else:
        return factor

scheduler = LambdaLR(calculate_gamma_initial_lr)
lrprint = LRPrinter(opt)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trial = Trial(model, opt, loss_function, callbacks=[scheduler, lrprint, save_state], metrics=['loss']).to(device)

trial.with_generators(train_loader)
trial.run(epochs=200)